In [1]:
import gym

In [2]:
env = gym.make('SpaceInvaders-v0')

In [3]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
    print('Episode: {}\nScore: {}'.format(episode, score))
    
env.close()

Episode: 1
Score: 185.0
Episode: 2
Score: 215.0
Episode: 3
Score: 95.0
Episode: 4
Score: 30.0
Episode: 5
Score: 255.0
Episode: 6
Score: 125.0
Episode: 7
Score: 210.0
Episode: 8
Score: 95.0
Episode: 9
Score: 125.0


In [4]:
#Import Neural Network Packages
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [11]:
del model

In [12]:
model = build_model(height, width, channels, actions)

In [8]:
#Importing keras-rl2 reinforcement learning functions
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [9]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [13]:
dqn = build_agent(model, actions)

In [14]:
dqn.compile(Adam(lr=0.001))

In [16]:
dqn.fit(env, nb_steps=40000, visualize=False, verbose=1)

Training for 40000 steps ...
Interval 1 (0 steps performed)
 1839/10000 [====>.........................] - ETA: 27:59 - reward: 0.3426done, took 378.658 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=10,visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


In [18]:
dqn.save_weights('models/dqn.h5f')

In [19]:
dqn.load_weights('models/dqn.h5f')